### Source code - https://github.com/iwiszhou/ML1010-final-project

This is a Yelp data-set. I would use this data-set to do a sentiment analysis.
I would build a model to predict the review either positive or negative.
This is a big data-set. Firstly, I would try to extra the review data and create a simple data-set,
which only contain Review & Rating. After that, I would create a new column which is Class.
Class column is either Positive or Negative. If Rating is grater than 3, I would mark Class to Positive. Otherwise,
Negative. If I have more time at the end, I would introduce one more value to Class column which is Neutral ( when
Rating is equal to 3 )

In [2]:
# Import libraries
import pandas as pd
import json
import numpy as np
import re
import nltk
import sqlite3
import matplotlib.pyplot as plt

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/iwiszhou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Read data from file
NOTE - the data-set is too big. I have already to several time, my computer crash. So that, I would start with first 10000 rows. I would increase the data-set size when training the model.

In [6]:
filename = "/Users/iwiszhou/Documents/Machine Learning/ML1010/ML1010-final-project/yelp_dataset/review.json"
rowCount = 0
rowLimit = 10000
df = []
with open(filename, 'r') as f:
    for line in f:
        df.append(json.loads(line))
        rowCount = rowCount + 1
        if rowCount > rowLimit:
            break
df = pd.DataFrame(df)

In [7]:
# Check data-set
print(df.head())

# Check number of rows
count_row = df.shape[0]
print(count_row)

# List all columns
print(df.columns)

                review_id                 user_id             business_id  \
0  Q1sbwvVQXV2734tPgoKj4Q  hG7b0MtEbXx5QzbzE6C_VA  ujmEBvifdJM6h6RLv4wQIg   
1  GJXCdrto3ASJOqKeVWPi6Q  yXQM5uF2jS6es16SJzNHfg  NZnhc2sEQy3RmzKTZnqtwQ   
2  2TzJjDVDEuAW6MR5Vuc1ug  n6-Gk65cPZL6Uz8qRm3NYw  WTqjgwHlXbSFevF32_DJVw   
3  yi0R0Ugj_xUx_Nek0-_Qig  dacAIZ6fTM6mqwW5uxkskg  ikCg8xy5JIg_NGPx-MSIDA   
4  11a8sVPMUFtaC7_ABRkmtw  ssoyf2_x0EQMed6fgHeMyQ  b1b1eb3uo-w561D0ZfCEiQ   

   stars  useful  funny  cool  \
0    1.0       6      1     0   
1    5.0       0      0     0   
2    5.0       3      0     0   
3    5.0       0      0     0   
4    1.0       7      0     0   

                                                text                 date  
0  Total bill for this horrible service? Over $8G...  2013-05-07 04:34:36  
1  I *adore* Travis at the Hard Rock's new Kelly ...  2017-01-14 21:30:33  
2  I have to say that this office really has it t...  2016-11-09 20:09:03  
3  Went in for a lunch. Steak sand

In [8]:
# Create a class column
def get_class_label_value(row):
    if row["stars"] >= 3:
        return "Positive"
    return "Negative"


df["class"] = df.apply(lambda row: get_class_label_value(row), axis=1)

In [9]:
# Create new data frame
filterDf = df[['business_id', 'text', 'class']]
print(filterDf.head().to_string())
print(filterDf.shape[0])
print(filterDf.columns)

              business_id                                               text     class
0  ujmEBvifdJM6h6RLv4wQIg  Total bill for this horrible service? Over $8G...  Negative
1  NZnhc2sEQy3RmzKTZnqtwQ  I *adore* Travis at the Hard Rock's new Kelly ...  Positive
2  WTqjgwHlXbSFevF32_DJVw  I have to say that this office really has it t...  Positive
3  ikCg8xy5JIg_NGPx-MSIDA  Went in for a lunch. Steak sandwich was delici...  Positive
4  b1b1eb3uo-w561D0ZfCEiQ  Today was my second out of three sessions I ha...  Negative
10001
Index(['business_id', 'text', 'class'], dtype='object')


In [10]:
# Text pre-processing
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')


def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc


normalize_corpus = np.vectorize(normalize_document)

In [11]:
# Create a clean text column

def get_clean_text_value(row):
    if row["text"]:
        return normalize_corpus(row["text"])
    return ""


filterDf["clean_text"] = df.apply(lambda row: get_clean_text_value(row), axis=1)

# Exploring Column Summaries
print(filterDf.sample(2, random_state=42).to_string())
print(filterDf.dtypes)
print(len(filterDf))
print(filterDf.describe(include=np.object).transpose().to_string())

/Users/iwiszhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


                 business_id                                               text     class                                         clean_text
6252  cSIt_NA2mQAmL_sk7f7TbA  I've been anticipating the opening of Thai Exp...  Positive  ive anticipating opening thai express quite so...
4684  GvKx1QL6XLyMZnWZun1DPQ  A great stop in Waxhaw. My wife and I frequent...  Positive  great stop waxhaw wife frequently come lazy sa...
business_id    object
text           object
class          object
clean_text     object
dtype: object
10001
             count unique                                                top  freq
business_id  10001   4619                             Wxxvi3LZbHNIDwJ-ZimtnA    79
text         10001  10001  I have no idea how the food tastes. We were he...     1
class        10001      2                                           Positive  7672
clean_text   10001  10001  man went brunch time 11am line looked long ( s...     1


In [12]:
# Save data to database
con = sqlite3.connect('yelp.db')
filterDf.to_sql("filter_reviews", con)